<a href="https://colab.research.google.com/github/OVP2023/E-CAP-2024/blob/main/%D0%A4%D0%BE%D1%80%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2_%D0%B8_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Формирование признаков и обучение модели

In [14]:
import pandas as pd
import numpy as np

In [81]:
train = pd.read_parquet('/content/drive/MyDrive/ecap/mydata.parquet')
train.head()

,variantid1,variantid2,target,Cosine_similarity_names,type1,art1,country1,brand1,type2,art2,country2,brand2
0,1447875869,1447872068,1,1.000000,[Свитер],None,[Япония],None,[Свитер],None,[Япония],None
1,1176231201,284733670,1,0.989477,[Фиксатор БДСМ],None,[Россия],[Джага Джага],[Фиксатор БДСМ],None,[Россия],[Джага Джага]
2,658617865,549848659,0,0.966868,[Корм сухой],[83809],[Нидерланды],[Hill's],[Корм сухой],[10039],[Чехия],[Hill's]
3,719320625,719370486,1,1.000000,[Бур],None,[Китай],[Elitech],[Бур],None,[Китай],[Elitech]
4,1067645658,949954740,0,0.986480,[Пенал],None,None,[Россия],[Пенал],None,[Россия],[Нет бренда]


In [82]:

for i in range(0, train.shape[0]):
    if train['art1'][i]==train['art2'][i] and train['art1'][i]!=None and train['art2'][i]!=None:
        train.loc[i,'yesart']=1
        train.loc[i,'noart']=0
    elif train['art1'][i]!=train['art2'][i] and train['art1'][i]!=None and train['art2'][i]!=None:
        train.loc[i,'noart']=1
        train.loc[i,'yesart']=0
    else:
        train.loc[i,'yesart']=0
        train.loc[i,'noart']=0


In [84]:
train = train.drop(columns=(['art1','art2','type1','type2','brand1','brand2','country1','country2']))

In [85]:
train.head()

,variantid1,variantid2,target,Cosine_similarity_names,yesart,noart
0,1447875869,1447872068,1,1.000000,0.0,0.0
1,1176231201,284733670,1,0.989477,0.0,0.0
2,658617865,549848659,0,0.966868,0.0,1.0
3,719320625,719370486,1,1.000000,0.0,0.0
4,1067645658,949954740,0,0.986480,0.0,0.0


In [94]:
#метод стохастического градиентного спуска и оценка точности модели
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

X=train[['Cosine_similarity_names','yesart','noart']] # признаки X
y=train['target'] # целевая переменная y

#Сформируйте X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

sgd_clf = SGDClassifier(random_state=42) # классификатор на основе метода стохастического градиентного спуска (англ. Stochastic Gradient Descent SGD)
sgd_clf.fit(X_train, y_train) # обучаем классификатор
# Для расчета матрицы ошибок сначала понадобится иметь набор прогнозов, чтобы их можно было сравнивать с фактическими целями
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
print(confusion_matrix(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))

[[424019    600]
 [367782  25559]]
0.5496332338989681


Пока правильность всего 55% на 2 признаках (артикул разбиваем на 2), нужно добавить еще 4) Признаки бренда, страны-изготовителя, типа товара, цвета